In [120]:
import pandas as pd
import numpy as np

In [162]:
df = pd.read_csv("./UH_2023/UH_2023_TRAIN.txt", sep = "|")
df = df.astype({"CAMPAÑA": str, "ID_FINCA": str, "ID_ZONA": str, "ID_ESTACION": str, "VARIEDAD": str, "MODO": str, "TIPO": str, "COLOR": str, "SUPERFICIE": str})

In [163]:
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
encoder = ce.OrdinalEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "VARIEDAD", "COLOR", "TIPO", "MODO"])

In [164]:
df_temporal = df.drop(axis = 1, columns = ["CAMPAÑA", "ALTITUD"])

df_encoded = encoder.fit_transform(df_temporal)

train = df_encoded[(df["PRODUCCION"] != 0) & (df_encoded["SUPERFICIE"] != "0.0")]

to_predict = df_encoded[~((df["PRODUCCION"] != 0) & (df_encoded["SUPERFICIE"] != 0))]
l_encoder = LabelEncoder()
train["SUPERFICIE"] = l_encoder.fit_transform(train["SUPERFICIE"])

to_predict = to_predict.drop(axis = 1, columns = ["SUPERFICIE"])

C:\Users\xavid\AppData\Local\Temp\ipykernel_9248\2113395605.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["SUPERFICIE"] = l_encoder.fit_transform(train["SUPERFICIE"])


In [165]:
df_supcat = train.groupby(by='SUPERFICIE').count().reset_index()
segur_train = df_supcat[df_supcat["TIPO"] == 1]["SUPERFICIE"]
train_nou = train[train['SUPERFICIE'].isin(list(segur_train))]

In [166]:
train_a_splitejar = train.drop(index=train_nou.index)

In [167]:
X_train_a_splitejar = train_a_splitejar.drop(axis = 1, columns = ["SUPERFICIE"])
y_train_a_splitejar = train_a_splitejar["SUPERFICIE"]

In [168]:
X_train_nou = train_nou.drop(axis = 1, columns = ["SUPERFICIE"])
y_train_nou = train_nou["SUPERFICIE"]

In [169]:
X_train_splitted, X_test, y_train_splitted, y_test = train_test_split(X_train_a_splitejar, y_train_a_splitejar, test_size=0.35, random_state=3, stratify = train_a_splitejar["SUPERFICIE"])

In [170]:
X_train = pd.concat([X_train_splitted, X_train_nou])
y_train = pd.concat([y_train_splitted, y_train_nou])

In [171]:
len(X_test) + len(X_train) == len(train)

True

In [ ]:
import xgboost as xgb
params_model = {'colsample_bytree': 0.4, 'gamma': 0.0, 'learning_rate': 0.15, 'max_depth': 60, 'min_child_weight': 1, 'n_estimators': 1000}
model = xgb.XGBClassifier(**params_model)
model.fit(X_train,y_train)
model.score(X_test, y_test)

In [ ]:
y_test_dec_gt = l_encoder.inverse_transform(y_test)
y_predicted = l_encoder.inverse_transform(model.predict(X_test))

In [ ]:
y_test_dec_gt = y_test_dec_gt.astype(np.float)
y_predicted = y_predicted.astype(np.float)

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test_dec_gt, y_predicted)